In [2]:
import pandas as pd
import numpy as np
import jqdatasdk as jq

In [3]:
jq.auth('13918852005','jxjslsc,.?2')

auth success （JQData现有流量增加活动，详情请咨询JQData管理员，微信号：JQData01）


In [6]:
tradeday = jq.get_trade_days(start_date='2009-01-01', end_date='2019-01-01', count=None)

In [7]:
tradedaylen = len(tradeday)

2431

In [128]:
# 沪深两市自2009年1月1日前上市股票列表
hslist = jq.get_index_stocks('000001.XSHG', date='2009-01-01')
sslist = jq.get_index_stocks('399106.XSHE', date='2009-01-01')
hslslist = hslist + sslist

In [163]:
cnt = 0
drop_cnt = 0
stock_num = len(hslslist) # 总股票数-未剔除有停牌
stock_lst = []

# 增量下载才开启，先检验是否已经下载
save_df = pd.read_csv('E:/Stock_Data/bin_data.csv',index_col=0)
code_lst = list(set(list(save_df['code'])))
cnt = int(len(save_df)/8)

for code in hslslist:
    
    # 增量下载才开启，先检验是否已经下载
    if code in code_lst:
        continue

    # 利润表
    incomestatement = jq.finance.run_query(
        jq.query(
            jq.finance.STK_INCOME_STATEMENT.code,
            jq.finance.STK_INCOME_STATEMENT.end_date,
            jq.finance.STK_INCOME_STATEMENT.operating_profit,
        ).filter(jq.finance.STK_INCOME_STATEMENT.code==code,
                 jq.finance.STK_INCOME_STATEMENT.end_date>='2009-12-31',
                 jq.finance.STK_INCOME_STATEMENT.end_date<='2018-01-01',
                 jq.finance.STK_INCOME_STATEMENT.source_id=='321003'
                ))
    ## 筛选至仅剩一年一份
    incomestatement['report_type'] = incomestatement['end_date'].apply(lambda x : str(x)[-5:])
    incomestatement = incomestatement[incomestatement['report_type']=='12-31']
    incomestatement = incomestatement.groupby(incomestatement['end_date']).max()[['code','operating_profit']]
    
    # 插入检验，是否存在停牌
    if len(incomestatement.index) != 9:
        cnt += 1
        drop_cnt += 1
        continue
    
    # 资产负债表
    balancesheet = jq.finance.run_query(
        jq.query(
            jq.finance.STK_BALANCE_SHEET.code,
            jq.finance.STK_BALANCE_SHEET.end_date,
            jq.finance.STK_BALANCE_SHEET.total_assets,
            jq.finance.STK_BALANCE_SHEET.total_owner_equities,
        ).filter(jq.finance.STK_BALANCE_SHEET.code==code,
                 jq.finance.STK_BALANCE_SHEET.end_date>='2009-12-31',
                 jq.finance.STK_BALANCE_SHEET.end_date<='2018-01-01',
                 jq.finance.STK_BALANCE_SHEET.source_id=='321003'
                ))
    ## 筛选至仅剩一年一份
    balancesheet['report_type'] = balancesheet['end_date'].apply(lambda x : str(x)[-5:])
    balancesheet = balancesheet[balancesheet['report_type']=='12-31']
    balancesheet = balancesheet.groupby(balancesheet['end_date']).max()[['total_assets','total_owner_equities']]
    
    # 估值表
    used_lst = ['market_cap','pe_ratio']
    valuation = pd.DataFrame(index=balancesheet.index,columns=used_lst)
    year_end_lst = [str(2009+i)+'-12-31' for i in range(9)]
    for i in range(9):
        year = str(2009+i)+'-12-31'
        valuation_inf = jq.get_fundamentals(
            jq.query(
                jq.valuation.market_cap,
                jq.valuation.pe_ratio
            ).filter(jq.valuation.code==code),year)      
        valuation.iloc[i,] = valuation_inf.iloc[0,]
        
    # 合并
    bin_df = pd.concat([incomestatement,balancesheet,valuation],axis=1)
    
    # 计算
    ## 投资分组
    bin_df['total_assets_1'] = bin_df['total_assets'].shift(1)
    bin_df['Inv'] = bin_df['total_assets']/bin_df['total_assets_1']-1
    ## 盈利能力分组
    bin_df['OP'] = bin_df['operating_profit']/ bin_df['total_owner_equities']
    ## 账面市值比分组
    bin_df['B_M'] = bin_df['total_owner_equities']/ (bin_df['market_cap']*100000000)
    ## 规模比分组
    bin_df['Size'] = bin_df['market_cap']
    
    # 生成时间列
    bin_df['date'] = bin_df.index
    bin_df = bin_df[['code','date','Size','B_M','OP','Inv','pe_ratio']].iloc[1:,:]
#     bin_df.index = list(range(8))
    
    if cnt == 0:
        save_df = bin_df
    else:
        save_df = pd.concat([save_df,bin_df],axis=0)
    
    cnt += 1
    stock_lst.append(code) # 成功运行的股票记录
    
    print('Complete:',cnt/stock_num*100,'% Total:',cnt-drop_cnt,end=' \r')
    
    if cnt%100 == 0:
        save_df.to_csv('E:/Stock_Data/bin_data.csv',index=True)
        pd.DataFrame(stock_lst).to_csv('E:/Stock_Data/stock_code.csv',index=False)

KeyboardInterrupt: 

In [157]:
save_df.to_csv('E:/Stock_Data/bin_data.csv',index=True)

In [158]:
valuation

,market_cap,pe_ratio
end_date,,
2009-12-31,149.006,37.7809
2010-12-31,100.098,15.4825
2011-12-31,103.222,8.1688
2012-12-31,132.102,7.7251
2013-12-31,150.999,8.283
2014-12-31,149.559,10.8755
2015-12-31,257.378,18.2685
2016-12-31,224.012,13.1732
2017-12-31,196.534,16.1268


In [148]:
jq.get_query_count()

{'total': 1000000, 'spare': 771347}

In [162]:
save_df

,code,date,Size,B_M,OP,Inv,pe_ratio
end_date,,,,,,,
2010-12-31,600000.XSHG,2010-12-31,1777.82,0.693433,0.203375,0.350457,10.0081
2011-12-31,600000.XSHG,2011-12-31,1583.68,0.944274,0.239109,0.225098,6.5307
2012-12-31,600000.XSHG,2012-12-31,1850.42,0.970907,0.247241,0.171719,5.5233
2013-12-31,600000.XSHG,2013-12-31,1759.02,1.17808,0.258282,0.169888,4.6439
2014-12-31,600000.XSHG,2014-12-31,2926.73,0.899588,0.234541,0.140158,6.3759
2015-12-31,600000.XSHG,2015-12-31,3407.99,0.934862,0.207367,0.202203,6.9003
2016-12-31,600000.XSHG,2016-12-31,3504.32,1.06421,0.186789,0.161153,6.4746
2017-12-31,600000.XSHG,2017-12-31,3695.43,1.16627,0.163057,0.047800,6.8044
2010-12-31,600004.XSHG,2010-12-31,101.43,0.666584,0.123237,0.005267,16.3239


In [159]:
valuation_inf

,market_cap,day,pe_ratio
0,196.5339,2017-12-29,16.1268


In [149]:
save_df

,code,date,Size,B_M,OP,Inv,pe_ratio
0,600000.XSHG,2010-12-31,1777.82,0.693433,0.203375,0.350457,10.0081
1,600000.XSHG,2011-12-31,1583.68,0.944274,0.239109,0.225098,6.5307
2,600000.XSHG,2012-12-31,1850.42,0.970907,0.247241,0.171719,5.5233
3,600000.XSHG,2013-12-31,1759.02,1.17808,0.258282,0.169888,4.6439
4,600000.XSHG,2014-12-31,2926.73,0.899588,0.234541,0.140158,6.3759
5,600000.XSHG,2015-12-31,3407.99,0.934862,0.207367,0.202203,6.9003
6,600000.XSHG,2016-12-31,3504.32,1.06421,0.186789,0.161153,6.4746
7,600000.XSHG,2017-12-31,3695.43,1.16627,0.163057,0.047800,6.8044
0,600004.XSHG,2010-12-31,101.43,0.666584,0.123237,0.005267,16.3239
1,600004.XSHG,2011-12-31,71.185,0.995803,0.134628,-0.057663,11.3671


In [64]:
incomestatement['report_type'] = incomestatement['end_date'].apply(lambda x : str(x)[-5:])
incomestatement = incomestatement[incomestatement['report_type']=='12-31']

In [74]:
[str(2009+i)+'-12-31' for i in range(9)]

['2009-12-31',
 '2010-12-31',
 '2011-12-31',
 '2012-12-31',
 '2013-12-31',
 '2014-12-31',
 '2015-12-31',
 '2016-12-31',
 '2017-12-31']

In [57]:
for code in hslist:
    
    # 利润表
    incomestatement = jq.finance.run_query(
        jq.query(
            jq.finance.STK_INCOME_STATEMENT,
        ).filter(jq.finance.STK_INCOME_STATEMENT.code==code,
                 jq.finance.STK_INCOME_STATEMENT.end_date>='2009-12-31',
                 jq.finance.STK_INCOME_STATEMENT.end_date<='2018-01-01',
                 jq.finance.STK_INCOME_STATEMENT.source_id=='321003'
                ))
#     incomestatement = incomestatement[incomestatement['end_date'][-5:]=='12-31']
    
    break

In [61]:
incomestatement.iloc[0:2,:]

,id,company_id,company_name,code,a_code,b_code,h_code,pub_date,start_date,end_date,...,commission_income,interest_expense,commission_expense,refunded_premiums,net_pay_insurance_claims,withdraw_insurance_contract_reserve,policy_dividend_payout,reinsurance_cost,non_current_asset_disposed,other_earnings
0,94284,420600000,上海浦东发展银行股份有限公司,600000.XSHG,600000,None,None,2010-04-07,2009-01-01,2009-12-31,...,None,None,None,None,None,None,None,None,None,None
1,94285,420600000,上海浦东发展银行股份有限公司,600000.XSHG,600000,None,None,2011-03-30,2009-01-01,2009-12-31,...,None,None,None,None,None,None,None,None,None,None
